In [1]:
import os
from os import path
import numpy as np
import random
import copy
import pandas as pd
import ipython_bell

from astropy.io import fits
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
from astropy.visualization import make_lupton_rgb
import sys

In [2]:
#Makes 3 plots per image: source simulation, lens cutout, complete simulation
def make_graphs(name, sim_i, sim_r, sim_g, cutout_i, cutout_r, cutout_g):
    loc = plticker.MultipleLocator(base=3.75)
    fig, (ax1, ax2, ax3) = plt.subplots(figsize=(15, 4), ncols=3)

    rgb = make_lupton_rgb(sim_i, sim_r, sim_g, Q=11., stretch=40.)
    ax1.set_title('Source(s)')
    ax1.xaxis.set_ticklabels([])
    ax1.yaxis.set_ticklabels([])
    #ax1.xaxis.set_major_locator(loc)
    #ax1.yaxis.set_major_locator(loc)
    #ax1.grid(which='major', axis='both', linestyle='-')
    original = ax1.imshow(rgb, aspect='equal')#, vmin=0, vmax = 250)
    #fig.colorbar(original, ax=ax1)

    rgb = make_lupton_rgb(cutout_i, cutout_r, cutout_g, Q=11., stretch=40.)
    ax2.set_title('Lens')
    ax2.xaxis.set_ticklabels([])
    ax2.yaxis.set_ticklabels([])
    #ax2.xaxis.set_major_locator(loc)
    #ax2.yaxis.set_major_locator(loc)
    #ax2.grid(which='major', axis='both', linestyle='-')
    cutout = ax2.imshow(rgb, aspect='equal')
    #fig.colorbar(cutout, ax=ax2)
            
    rgb = make_lupton_rgb(sim_i+cutout_i, sim_r+cutout_r, sim_g+cutout_g, Q=11., stretch=40.)
    ax3.set_title('Complete Simulation')
    ax3.xaxis.set_ticklabels([])
    ax3.yaxis.set_ticklabels([])
    #ax3.xaxis.set_major_locator(loc)
    #ax3.yaxis.set_major_locator(loc)
    #ax3.grid(which='major', axis='both', linestyle='-')
    projection = ax3.imshow(rgb, aspect='equal')
    #fig.colorbar(projection, ax=ax3)
    sub_path = '/Users/jimenagonzalez/research/DSPL/Simulations-Double-Source-Gravitational-Lensing/'
    #plt.savefig(sub_path + 'Data/Sim_complete/Image' + name + '.png', bbox_inches='tight')
    plt.show()
    

In [3]:
# Writes fit file
path_doc = '/Users/jimenagonzalez/research/DSPL/ParameterSpaceSimulations/Data/Simu_complete/'
def write_fit_file(c_folder, exp, x):
    if not os.path.exists(path_doc + c_folder):
        os.makedirs(path_doc + c_folder)
    primary = fits.PrimaryHDU()
    image = fits.ImageHDU(x, name="IMAGE")
    hdu_list = fits.HDUList([primary, image])
    hdu_list.writeto(path_doc +c_folder + exp + '.fits', overwrite=True)

In [4]:
directory_sim = '/Users/jimenagonzalez/research/DSPL/ParameterSpaceSimulations/Data/Simu/'
directory_cutouts = '/Users/jimenagonzalez/research/DSPL/Simulations-Double-Source-Gravitational-Lensing/Data/lens_cutouts/'
def complete_simulations(c_folder, exp, m, ids):
    end_name = ['_g.fits', '_i.fits', '_r.fits']
    x = np.zeros((1,3,46,46)) #Complete Simulations
    n = 0
    for lens_id in ids:
        path_sim_id = directory_sim + c_folder + exp + '/' + str(lens_id)
        if(n >= m):
            break
        #check that the sim id folder exist (if not continue)
        if(os.path.exists(path_sim_id) == False):
            continue
        num_sim_id = int(len(os.listdir(path_sim_id))/3) # num sim with same lens
        #loop on each simulation produced using the same lens
        for j in range(1, num_sim_id+1):
            n += 1 #number of total simulations
            path_cutout = directory_cutouts + str(lens_id) 
            path_sim = path_sim_id + '/' + str(j)
            if(os.path.exists(path_cutout + end_name[0]) == False): 
                continue
            with fits.open(path_cutout + end_name[0]) as hdul: cutout_g = hdul[0].data
            with fits.open(path_cutout + end_name[1]) as hdul: cutout_i = hdul[0].data
            with fits.open(path_cutout + end_name[2]) as hdul: cutout_r = hdul[0].data
            with fits.open(path_sim + end_name[0]) as hdul: sim_g = hdul[0].data
            with fits.open(path_sim + end_name[1]) as hdul: sim_i = hdul[0].data
            with fits.open(path_sim + end_name[2]) as hdul: sim_r = hdul[0].data
            
            #make_graphs(str(lens_id), sim_i, sim_r, sim_g, cutout_i, cutout_r, cutout_g)
            ele_sim = np.array([cutout_g + sim_g, cutout_r + sim_r, cutout_i + sim_i])
            ele_sim = np.rot90(ele_sim, k=random.randint(0, 4), axes=(1, 2))
            if(random.randint(0, 1) == 1):
                ele_sim = np.flip(ele_sim, 2)
            x = np.append(x, [ele_sim], axis = 0)
    x = np.delete(x, 0, axis = 0)
    return(x)


In [5]:
c_folder = 'Exp_C:0.0-0.2/'
def write_complete(c_folder):
    for i in range(100):
        exp = i+1
        ids = pd.read_csv('Data/all_data_sim.csv')['COADD_OBJECT_ID'].sample(frac = 1)
        x = complete_simulations(c_folder, str(exp), 20000, ids)
        write_fit_file(c_folder, str(exp), x)
        print(exp, len(x))

In [6]:
c_folder = 'Exp_C:0.8-1.0/'
write_complete(c_folder)

1 65
2 126
3 227
4 436
5 599
6 600
7 506
8 230
9 66
10 10
11 64
12 135
13 350
14 599
15 600
16 598
17 600
18 375
19 73
20 17
21 81
22 140
23 330
24 600
25 599
26 600
27 599
28 404
29 106
30 25
31 44
32 129
33 327
34 599
35 600
36 599
37 599
38 304
39 87
40 18
41 30
42 92
43 296
44 600
45 598
46 600
47 599
48 228
49 48
50 11
51 21
52 53
53 202
54 600
55 600
56 599
57 563
58 155
59 29
60 5
61 11
62 23
63 106
64 422
65 600
66 600
67 304
68 100
69 28
70 5
71 9
72 16
73 60
74 246
75 479
76 458
77 166
78 50
79 12
80 4
81 3
82 5
83 25
84 95
85 237
86 217
87 89
88 21
89 7
90 5
91 0
92 1
93 15
94 57
95 120
96 111
97 40
98 10
99 3
100 0


In [7]:
bell -n notify print('hello')

hello


In [8]:
#bell -n say write_fit_file(c_folder, exp, x)